分为两步：  
* zz2dis  
* dis2ks  

In [1]:
import sys
sys.path.append('/home/ian/code/github')

In [2]:
from utils.pymongo_wrapper import pymongo_wrapper

In [3]:
pmw = pymongo_wrapper()

In [4]:
zztable = pmw.get_collection('jiankang39','zznew')

In [6]:
zzlist = list(pmw.find_all(zztable,fieldlist=['症状名称','相关疾病']))
zzlist[:1]

[{'症状名称': '多梦',
  '相关疾病': ['睡眠障碍',
   '缺血缺氧性脑病',
   '脑外伤伴发的精神障碍',
   '癔症',
   '慢性乙肝',
   '慢性疲劳综合症',
   '体质性低血压',
   '女子梦交',
   '肝火上炎证',
   '肝阳上亢',
   '阴虚火旺',
   '阴虚发热',
   '心阴虚',
   '心血虚',
   '结肠炎',
   '黧黑斑',
   '原发性高血压',
   '十二指肠胃反流及胆汁反流性胃炎',
   '卵巢早衰',
   '盆腔淤血综合征',
   '酒中毒性周围神经病',
   '男性更年期综合症',
   '汞中毒',
   '纤维肌痛综合征',
   '肝炎后综合征',
   '复发性阿弗他性口腔炎',
   '心脏神经官能症',
   '失眠症',
   '焦虑症',
   '老年人神经症',
   '工业中毒所致的精神障碍',
   '心血管疾病伴发的精神障碍',
   '慢性硬脑膜下血肿']}]

In [7]:
zzs = list({i['症状名称'] for i in zzlist})
len(zzs)

3488

In [29]:
diseases1 = set()
for i in zzlist:
    diseases1 = diseases1.union(i['相关疾病'])
diseases1 = list(diseases1)
len(diseases1)

6482

### 去掉极严重的病  
罕见程度 == 0  or 不在疾病库中

In [26]:
distable = pmw.get_collection('jiankang39','diseases')

In [27]:
r = set()

In [31]:
for i in diseases1:
    d = pmw.find_one(distable,{'疾病名称':i},fieldlist=['罕见程度'])
    if d:
        if d['罕见程度'] == 1 or d['罕见程度'] == 2:
            r.add(i)
    else:
        print(i)
        r.add(i)
len(r)

性厌恶
腺瘤
胸腺小细胞癌
死精症
男性假两性畸形
痞气
光气中毒
小儿触电与雷击
精瘀证
消化系统溃疡病
无精症
产后风
副球孢子菌病
女性生殖道多部位原发癌
内伤发热
乙状结肠间疝
苯海拉明中毒
小儿单纯性血尿
骨纤维肉瘤
蓐劳
增生性骨关节病
小儿其他沙门菌感染
特发性高嗜酸性细胞综合征
骨软骨瘤
肤胀
女性的青春期延迟
精索恶性肿瘤
慢性肠炎
淋病合并症精囊炎
带下病
男性生殖器外伤
真两性畸形
菌血症
真菌性食物中毒
鱼胆中毒
空调病
代谢性酸中毒
组织胞浆菌病
节育手术并发症
斑痧
回归热
慢性附件炎
疝气
老年人骨质疏松
全植物神经功能不全
水臌
中满
消中
脐疝
婴儿脐疝
骨母细胞瘤
对乙酰氨基酚中毒
软骨发育异常
不射精症
夏季低热
进行性骨干发育不全
骨雅司病
男性生殖腺机能减退症
骨囊肿
妇女多毛症
婴幼儿脐疝
纹状骨瘤
四氯化碳中毒
黄曲霉素中毒
痢疾
痃癖
多发性骨骺发育异常
转移性胸膜肿瘤
生长期脱发
骨化性肌炎
女性慢性淋病
软组织肿瘤
白线疝
小棘毛壅病
早产
淋球菌性腹膜炎
宿醉
性早熟
下消
造口旁疝
乳糜胸
腰骶部脂肪疝
内生性软骨瘤
黏多糖贮积症Ⅶ型
胁痛
黏多糖贮积症Ⅳ型
假斑秃
妊娠合并生殖器疱疹
男科外伤
产后虚羸
朗格汉斯组织细胞增多症
生长痛
急性脓胸
血热崩漏
胸腺瘤
反分化软骨肉瘤
胸膜炎
念珠状发
骨炎
气胀
拉休尤-格拉哈姆-利特尔综合征
子满
小儿两性畸形
经行头痛
妇女雄激素源性秃发
腹部创伤
尿精
坐月子
Q热
隆突性皮肤纤维肉瘤
妊娠合并支原体感染
腹膜后脂肪瘤
腹卒中综合征
脐部脐肠系膜管息肉
生乳头汗管囊腺瘤
经行发热
风邪致病
维生素中毒
C5功能不全综合征
继发性痛经
精囊囊肿
子宫复旧不全
小儿疳痢
下背部痛
急性腰扭伤
少腹如扇
脓胸
性交疼痛
痛性肥胖病
锌缺乏病
不孕症
男性不育
妊娠合并生殖道沙眼衣原体感染
腹腔室隔综合征
精囊恶性肿瘤
软骨肉瘤
点状骨骺发育异常
男性生殖器畸形
颈部软组织损伤
小儿46-XY单纯性腺发育不全综合征
盆腔炎性肿块
食物不耐受
淋病
寄生虫病
房事两感症
真菌中毒症
新生儿B链球菌感染
小儿生殖细胞瘤
阴盛阳衰症
虚劳
腰疝
月经失调性不孕症
小儿泄泻
腹绞痛综合征
毛囊性黏蛋白病
五氯酚钠中毒
骨嗜酸性肉芽肿
进行性骨干发育不良
膀胱上内疝
腰肌劳损

980

In [34]:
diseases = [i for i in diseases1 if i not in r]
len(diseases)

5502

In [9]:
from utils.pandas_wrapper import pandas_wrapper as pdw 

In [10]:
zzseries = pdw.build_series(zzs)
zzseries[:5]

0          指甲翘起
1       扁桃体充血肥大
2      拇指不明原因疼痛
3          咽喉疼痛
4    头或颈部发胀及跳动感
dtype: object

In [33]:
disseries = pdw.build_series(diseases)
disseries[:5]

0       肝动脉闭塞
1    急性化脓性腮腺炎
2        赤脉传睛
3       肛门瘙痒症
4        支气管炎
dtype: object

In [12]:
zzseries_inv = pdw.build_series(zzseries.index,index=zzseries.values)
zzseries_inv[:5]

指甲翘起          0
扁桃体充血肥大       1
拇指不明原因疼痛      2
咽喉疼痛          3
头或颈部发胀及跳动感    4
dtype: int64

In [35]:
disseries_inv = pdw.build_series(disseries.index,index=disseries.values)
disseries_inv[:5]

肝动脉闭塞       0
急性化脓性腮腺炎    1
赤脉传睛        2
肛门瘙痒症       3
支气管炎        4
dtype: int64

### 构建疾病症状矩阵

In [40]:
from utils.numpy_wrapper import numpy_wrapper as npw

zzdismatr = npw.build_zeros_array((len(zzseries),len(disseries)))

for i in zzlist:
    x = zzseries_inv[i['症状名称']]
    for ii in i['相关疾病']:
        if ii in disseries_inv:
            zzdismatr[x,disseries_inv[ii]] = 1

diszzmatr = zzdismatr.T

In [20]:
zz1 = ['指甲翘起']
temp = npw.build_zeros_array((1,len(zzseries)))[0]
for i in zz1:
    temp[zzseries_inv[i]] = 1

In [21]:
from math import sqrt
def cal_cos_similarity(a1,a2):
    a = a1.dot(a2)
    b = a1.dot(a1)
    c = a2.dot(a2)
    if b == 0 or c==0:
        return 0
    return a/sqrt( b*c )

In [39]:
diszzmatr[1]

array([0., 0., 0., ..., 0., 0., 0.])

In [44]:
diszzmatr.shape

(5502, 3488)

In [23]:
temp.shape

(3488,)

In [49]:
#输出可能的疾病和概率
def zz2disease(zzs,gender='男',age='4',other='普通'):
    genderdict = {'男':0,'女':1}
    otherdict = {'普通':[0],'孕期':[1],'产褥期':[2],'孕妇及产褥期':[1,2]}
    gender = genderdict[gender]
    other = otherdict[other]
    temp = npw.build_zeros_array((1,len(zzseries)))[0]
    for i in zzs:
        temp[zzseries_inv[i]] = 1
    l1 = list()
    for i in range(len(disseries)):
        l1.append(cal_cos_similarity(temp,diszzmatr[i]))
    s1 = pdw.build_series(l1)
    s2 = s1.sort_values(ascending=False)
    diss = list(disseries[s2.iloc[s2.nonzero()].index].values)
    dissprob = list(s2.iloc[s2.nonzero()].values)
    for i in range(len(diss)):
        d = pmw.find_one(distable,{'疾病名称':diss[i],'限定性别':{'$in':[gender]},'限定年龄':{'$in':[age]},'备注':{'$in':other}},fieldlist=['罕见程度','多发年龄','多发性别'])
        if d:
            if d['罕见程度'] == 1:
                dissprob[i] *= 0.5
            if age not in d['多发年龄']:
                dissprob[i] *= 0.5
            if gender not in d['多发性别']:
                dissprob[i] *= 0.5
        else:
            dissprob[i] = 0
    return diss, dissprob

In [48]:
zz1 = ['牙龈出血']
zz2disease(zz1)

(['血小板无力症',
  'α-贮存池病',
  '严重肝病引起获得性凝血因子异常',
  '血小板型假性血管性血友病',
  '登革出血热',
  'δ-贮存池病',
  '药物免疫性血小板减少性紫癜',
  '真性红细胞增多症',
  '黑热病',
  '外阴-阴道-牙龈综合征',
  '白血病',
  '再生障碍性贫血',
  '血管性血友病',
  '牙宣',
  '小儿巨大血小板综合征',
  '小儿血小板释放功能缺陷性疾患',
  '老年人肝硬化',
  '小儿血小板无力症',
  '小儿血友病',
  '轻链病与轻链沉积病',
  '肥大性龈炎',
  '维生素K缺乏病',
  '急性失血所致贫血',
  '小儿出血性疾病',
  '小儿高黏稠综合征',
  '创伤性口炎',
  '暴发性肝炎样综合征',
  '小儿继发性免疫性血小板减少性紫癜',
  '急性白血病',
  '小儿肾性贫血',
  '汞中毒',
  '肝纤维化',
  '小儿特发性血小板减少性紫癜',
  '特发性血小板减少性紫癜',
  '小儿过敏性紫癜',
  '小儿急性髓样白血病',
  '坏血病',
  '牙周炎',
  '老年人慢性肾功能衰竭',
  '牙周病'],
 [1.0,
  0.7071067811865475,
  0.7071067811865475,
  0.7071067811865475,
  0.7071067811865475,
  0.7071067811865475,
  0.7071067811865475,
  0.5773502691896258,
  0.5773502691896258,
  0.5773502691896258,
  0.5773502691896258,
  0.5773502691896258,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.4472135954999579,
  0.4472135954999579,
  0.4472135954999579,
  0.4472135954999579,
  0.4472135954999579,
  0.4082482904638631,
  0.4082482904638631,
  0

In [50]:
zz1 = ['牙龈出血']
zz2disease(zz1)

(['血小板无力症',
  'α-贮存池病',
  '严重肝病引起获得性凝血因子异常',
  '血小板型假性血管性血友病',
  '登革出血热',
  'δ-贮存池病',
  '药物免疫性血小板减少性紫癜',
  '真性红细胞增多症',
  '黑热病',
  '外阴-阴道-牙龈综合征',
  '白血病',
  '再生障碍性贫血',
  '血管性血友病',
  '牙宣',
  '小儿巨大血小板综合征',
  '小儿血小板释放功能缺陷性疾患',
  '老年人肝硬化',
  '小儿血小板无力症',
  '小儿血友病',
  '轻链病与轻链沉积病',
  '肥大性龈炎',
  '维生素K缺乏病',
  '急性失血所致贫血',
  '小儿出血性疾病',
  '小儿高黏稠综合征',
  '创伤性口炎',
  '暴发性肝炎样综合征',
  '小儿继发性免疫性血小板减少性紫癜',
  '急性白血病',
  '小儿肾性贫血',
  '汞中毒',
  '肝纤维化',
  '小儿特发性血小板减少性紫癜',
  '特发性血小板减少性紫癜',
  '小儿过敏性紫癜',
  '小儿急性髓样白血病',
  '坏血病',
  '牙周炎',
  '老年人慢性肾功能衰竭',
  '牙周病'],
 [1.0,
  0.7071067811865475,
  0.35355339059327373,
  0.7071067811865475,
  0.35355339059327373,
  0.7071067811865475,
  0.7071067811865475,
  0.5773502691896258,
  0.5773502691896258,
  0,
  0.2886751345948129,
  0.2886751345948129,
  0.5,
  0.5,
  0,
  0,
  0,
  0,
  0,
  0.5,
  0.4472135954999579,
  0.4472135954999579,
  0.4472135954999579,
  0,
  0,
  0,
  0.4082482904638631,
  0,
  0.4082482904638631,
  0,
  0.1889822365046136,
  0.3779644730092272,
  0

In [119]:
zz1 = ['恶心','头痛','乏力','健忘']
zz2disease(zz1)

(['蛋白胨性荨麻疹',
  '慢性鼻咽炎',
  '鼻咽炎',
  '功能性呕吐',
  '空调病',
  '胃内异物',
  '贾第虫病',
  '宿醉',
  '药物中毒',
  '胰腺肿瘤',
  '颈部软组织损伤',
  '头风病',
  '溢饮',
  '腮腺炎',
  '中性粒细胞减少症',
  '慢性粒细胞白血病',
  '急性无黄疸型肝炎',
  '阑尾炎',
  '中老年男子部分性雄激素缺乏综合征',
  '原发性中枢神经系统淋巴瘤',
  '寒冷性荨麻疹',
  '老年人血管性痴呆',
  '房事头疼症',
  '上颌窦炎',
  '特发性室性心动过速',
  '原发性高血压',
  '肝阳上亢',
  '肾小管间质性肾炎葡萄膜炎综合征',
  '原发性下肢深静脉瓣膜关闭不全',
  '电脑视觉综合症',
  '溃疡病出血',
  '风温',
  '慢性荨麻疹',
  '密码综合症',
  '慢性白血病',
  '职业枯竭',
  '寒湿泄泻',
  'Castleman病',
  '夏季低热',
  '糖尿病并结核病',
  '肾精亏虚',
  '旅途精神病',
  '急性上颌窦炎',
  '葡萄球菌感染',
  '胆热',
  '慢性乳腺炎',
  '急性糜烂性胃炎',
  '自身红细胞过敏性紫癜',
  '小儿流涎症',
  '慢性乙肝',
  '性功能异常伴发的精神障碍',
  '女子梦交',
  '阴挺',
  '老年人高脂血症',
  '性病相关性胃肠道感染',
  '阳明头痛',
  '电视眼病',
  '心虚',
  '休息痢',
  '一氧化碳中毒',
  '小儿暑温',
  '红细胞增多症',
  '生长痛',
  '腔隙性脑梗死',
  '淋巴细胞白血病',
  '肝热病',
  '风热头痛',
  '疫毒痢',
  '性交晕厥',
  '病毒性支气管炎',
  '糙皮病',
  '传染性淋巴细胞增多症',
  '镉中毒',
  '老年痴呆症',
  '淋病后综合征',
  '虚寒咳嗽',
  '瘀血头痛',
  '原发性骨髓纤维化',
  '血虚头痛',
  '脑萎',
  '肺螨病',
  '成人T细胞白血病',
  '杨梅疮',
  '老年人低血糖昏迷',
  '空肠、回肠憩室',
  '急性细菌性前列腺炎',
  

In [51]:
zz1 = ['小腿酸痛','小腿发胀','脚后跟疼']
zz2disease(zz1)

(['情感性交叉擦腿症',
  '原发性下肢静脉瓣膜关闭不全',
  '血栓闭塞性脉管炎',
  '下肢静脉血栓',
  '老年静脉血栓症',
  '腰腿痛',
  '急性单纯外伤性腰腿痛',
  '慢性单纯外伤性腰腿痛'],
 [0.3333333333333333,
  0.2886751345948129,
  0.2581988897471611,
  0.2581988897471611,
  0,
  0.20412414523193154,
  0,
  0.1543033499620919])

In [124]:
distable = pmw.get_collection('jiankang39','diseases')

In [54]:
def zz2ks(zzs,gender='男',age='4',other='普通'):
    a,b = zz2disease(zzs,gender,age,other)
    ksdict = dict()
    for i in range(len(a)):
        ks = distable.find_one({'疾病名称':a[i]})
        if ks and '挂号的科室' in ks:
            ks = ks['挂号的科室'].split()
            print(a[i],ks,b[i])
            for ii in ks:
                if ii in ksdict:
                    ksdict[ii] += b[i]
                else:
                    ksdict[ii] = b[i]
    return pdw.build_series(ksdict).sort_values(ascending=False)

In [96]:
list(disseries[s2.iloc[s2.nonzero()].index].values)

['颅内动脉瘤', '小儿甲状旁腺功能减退症']

In [55]:
zz2ks(['小腿酸痛','小腿发胀','脚后跟疼'])

情感性交叉擦腿症 ['儿科'] 0.3333333333333333
原发性下肢静脉瓣膜关闭不全 ['血管外科'] 0.2886751345948129
血栓闭塞性脉管炎 ['血管外科'] 0.2581988897471611
下肢静脉血栓 ['血管外科'] 0.2581988897471611
老年静脉血栓症 ['血管外科', '老年科'] 0
腰腿痛 ['老年科', '普外科'] 0.20412414523193154
急性单纯外伤性腰腿痛 ['外伤科', '骨科'] 0
慢性单纯外伤性腰腿痛 ['骨科'] 0.1543033499620919


血管外科    0.805073
儿科      0.333333
普外科     0.204124
老年科     0.204124
骨科      0.154303
外伤科     0.000000
dtype: float64

In [56]:
zz2ks(['恶心','头痛','乏力','健忘'])

蛋白胨性荨麻疹 ['皮肤科'] 0.7071067811865475
鼻咽炎 ['耳鼻喉科'] 0.5303300858899106
慢性鼻咽炎 ['耳鼻喉科'] 0.5303300858899106
胃内异物 ['消化内科'] 0.5
原发性中枢神经系统淋巴瘤 ['脑外科', '肿瘤科'] 0.25
胰腺肿瘤 ['肿瘤科', '肝胆外科'] 0.4472135954999579
药物中毒 ['急诊科', '消化内科'] 0.22360679774997896
急性无黄疸型肝炎 ['消化内科', '传染科'] 0.4082482904638631
阑尾炎 ['消化内科', '胃肠外科'] 0.35355339059327373
头风病 ['中医科'] 0.35355339059327373
贾第虫病 ['消化内科', '传染科'] 0.17677669529663687
腮腺炎 ['内分泌科', '口腔科'] 0.17677669529663687
功能性呕吐 ['消化内科', '内科'] 0.35355339059327373
上颌窦炎 ['耳鼻喉科'] 0.35355339059327373
Castleman病 ['血液科', '内分泌科'] 0.35355339059327373
溢饮 ['中医科'] 0
中性粒细胞减少症 ['血液科'] 0.35355339059327373
特发性室性心动过速 ['心血管内科'] 0.2886751345948129
原发性骨髓纤维化 ['血液科'] 0.2886751345948129
慢性荨麻疹 ['皮肤科'] 0.2886751345948129
房事头疼症 ['中医科'] 0.2886751345948129
原发性高血压 ['心血管内科'] 0.2886751345948129
肝阳上亢 ['中医科'] 0.2886751345948129
慢性乳腺炎 ['妇科'] 0.14433756729740646
慢性白血病 ['血液科'] 0.14433756729740646
葡萄球菌感染 ['内科', '传染科'] 0.2886751345948129
中老年男子部分性雄激素缺乏综合征 ['内分泌科', '男科'] 0
溃疡病出血 ['消化内科', '胃肠外科'] 0.2886751345948129
淋巴细胞白

中医科      6.589440
消化内科     5.628383
血液科      2.506258
传染科      2.056509
呼吸内科     1.856835
耳鼻喉科     1.664214
心理咨询     1.554948
肿瘤科      1.489318
皮肤科      1.422559
内分泌科     1.360430
心血管内科    1.275339
内科       1.235672
眼科       1.212310
肝胆外科     1.193026
性病科      0.790657
胃肠外科     0.642229
男科       0.589366
神经内科     0.478618
营养科      0.454124
肝炎       0.416667
急诊科      0.406486
脑外科      0.391737
儿科       0.367944
普外科      0.353553
肾内科      0.352690
血管外科     0.344491
妇科       0.333320
口腔科      0.321114
心胸外科     0.275965
肝病       0.250000
精神病科     0.204124
体检保健科    0.203087
泌尿外科     0.176777
肛肠外科     0.176777
风湿科      0.125000
老年科      0.058926
产科       0.000000
dtype: float64

In [57]:
zz2ks(['牙龈出血'])

血小板无力症 ['血液科'] 1.0
α-贮存池病 ['血液科'] 0.7071067811865475
严重肝病引起获得性凝血因子异常 ['血液科'] 0.35355339059327373
血小板型假性血管性血友病 ['血液科'] 0.7071067811865475
登革出血热 ['传染科', '急诊科'] 0.35355339059327373
δ-贮存池病 ['血液科'] 0.7071067811865475
药物免疫性血小板减少性紫癜 ['血液科'] 0.7071067811865475
真性红细胞增多症 ['血液科'] 0.5773502691896258
黑热病 ['传染科'] 0.5773502691896258
外阴-阴道-牙龈综合征 ['妇科', '口腔科'] 0
白血病 ['血液科', '肿瘤科'] 0.2886751345948129
再生障碍性贫血 ['血液科'] 0.2886751345948129
血管性血友病 ['血液科'] 0.5
牙宣 ['口腔科', '中医科'] 0.5
小儿巨大血小板综合征 ['儿科', '血液科'] 0
小儿血小板释放功能缺陷性疾患 ['儿科', '血液科'] 0
老年人肝硬化 ['老年科', '肝胆外科'] 0
小儿血小板无力症 ['儿科', '血液科'] 0
小儿血友病 ['儿科', '血液科'] 0
轻链病与轻链沉积病 ['血液科', '肾内科'] 0.5
肥大性龈炎 ['口腔科'] 0.4472135954999579
维生素K缺乏病 ['血液科', '内科'] 0.4472135954999579
急性失血所致贫血 ['血液科', '内科'] 0.4472135954999579
小儿出血性疾病 ['儿科', '血液科'] 0
小儿高黏稠综合征 ['儿科', '血液科'] 0
创伤性口炎 ['口腔科', '外伤科'] 0
暴发性肝炎样综合征 ['肝炎', '传染科'] 0.4082482904638631
小儿继发性免疫性血小板减少性紫癜 ['儿科', '血液科'] 0
急性白血病 ['血液科'] 0.4082482904638631
小儿肾性贫血 ['儿科', '血液科'] 0
汞中毒 ['急诊科'] 0.1889822365046136
肝纤维化 ['肝病', '消化内科'] 0.377964

血液科     7.992910
口腔科     1.368712
传染科     1.339152
内科      1.247981
急诊科     0.542536
中医科     0.500000
肾内科     0.500000
肝炎      0.408248
肝病      0.377964
消化内科    0.377964
营养科     0.316228
儿科      0.316228
肿瘤科     0.288675
妇科      0.000000
肝胆外科    0.000000
外伤科     0.000000
老年科     0.000000
dtype: float64

In [63]:
zz2ks(['男子性功能障碍'])

腹主动脉血栓形成综合征 ['消化内科', '血管外科'] 0.35355339059327373
血色病性心肌病 ['心血管内科', '心胸外科'] 0.7071067811865475
早泄 ['男科', '生殖健康'] 0.5773502691896258
继发性早泄 ['男科', '泌尿外科'] 0.5773502691896258
原发性早泄 ['男科', '泌尿外科'] 0.5773502691896258
小儿雄激素不敏感综合征 ['儿科', '内分泌科'] 0
阳痿 ['男科', '生殖健康'] 0.5773502691896258
阴茎异常勃起 ['男科', '泌尿外科'] 0.5773502691896258
老年男性性功能障碍 ['男科', '老年科'] 0
双阴茎 ['男科', '泌尿外科'] 0
肝硬化男性性功能减弱综合征 ['消化内科', '男科'] 0.5
第Ⅱ、Ⅲ型脊髓血管畸形 ['心血管内科', '血管外科'] 0.5
肾精亏虚 ['中医科'] 0.4472135954999579
小阴茎 ['男科'] 0.4082482904638631
维生素E缺乏病 ['内科', '儿科'] 0.4082482904638631
克兰费尔特综合征 ['内分泌科', '中医科'] 0.4082482904638631
小儿松果体区肿瘤 ['脑外科', '肿瘤科'] 0
围术期伴发的精神障碍 ['心理咨询', '精神病科'] 0.4082482904638631
精囊炎 ['男科', '泌尿外科'] 0.4082482904638631
慢性细菌性前列腺炎 ['泌尿外科', '男科'] 0.3779644730092272
先天性睾丸发育不全 ['泌尿外科', '男科'] 0.1889822365046136
细菌性前列腺炎 ['男科', '泌尿外科'] 0.35355339059327373
Shy-Drager综合征 ['神经内科', '血液科'] 0.35355339059327373
阴部神经干痛 ['针灸科', '神经内科'] 0.35355339059327373
肥胖性生殖无能综合征 ['内分泌科'] 0.3333333333333333
无功能垂体腺瘤 ['内分泌科'] 0.16666666666666666
睡眠呼吸暂停综合征 [

男科       5.123748
泌尿外科     3.060799
心血管内科    1.207107
生殖健康     1.154701
中医科      1.084878
内分泌科     1.052586
消化内科     0.853553
血管外科     0.853553
心胸外科     0.707107
神经内科     0.707107
儿科       0.408248
内科       0.408248
心理咨询     0.408248
精神病科     0.408248
血液科      0.353553
针灸科      0.353553
呼吸内科     0.316228
耳鼻喉科     0.316228
肾内科      0.229416
性病科      0.000000
老年科      0.000000
脑外科      0.000000
肿瘤科      0.000000
dtype: float64

In [59]:
zz2ks(['头胀','单侧头痛'])

老年人颅内压增高 ['神经内科', '脑外科'] 0
瘀血头痛 ['中医科'] 0.5345224838248488
老年人正常压力脑积水 ['脑外科', '老年科'] 0
小儿偏头痛 ['儿科', '神经内科'] 0
颅内高压综合征 ['神经内科', '脑外科'] 0.10206207261596577
老年人原发性恶性高血压引起的恶性小动脉肾硬化 ['肾内科', '老年科'] 0
小儿高血压 ['心血管内科', '儿科'] 0.35355339059327373
颅骨结核 ['脑外科', '传染科'] 0.17677669529663687
肝炎后综合征 ['肝炎'] 0.35355339059327373
老年人上腔静脉受压综合征 ['心血管内科', '老年科'] 0
偏头痛 ['神经内科'] 0.15811388300841897
颅内动脉瘤 ['肿瘤科', '脑外科'] 0.036084391824351615
老年人高脂血症 ['心血管内科', '老年科'] 0
高血压 ['心血管内科', '心胸外科'] 0
老年人高血压 ['心血管内科', '老年科'] 0
小儿后颅窝型脑积水综合征 ['脑外科', '儿科'] 0
高颅压性脑积水 ['脑外科', '神经内科'] 0.125
夏季感冒 ['中医科', '呼吸内科'] 0.23570226039551587
新生儿颅内出血 ['儿科', '脑外科'] 0
小儿颅内肿瘤 ['神经内科', '脑外科'] 0
腹型偏头痛综合征 ['消化内科', '神经内科'] 0.21320071635561041
原发性高血压 ['心血管内科'] 0.20412414523193154
鼻咽癌 ['肿瘤科', '耳鼻喉科'] 0.1889822365046136
痰浊头痛 ['中医科', '呼吸内科'] 0.1543033499620919


中医科      0.924528
神经内科     0.598377
心血管内科    0.557678
脑外科      0.439923
呼吸内科     0.390006
肝炎       0.353553
儿科       0.353553
肿瘤科      0.225067
消化内科     0.213201
耳鼻喉科     0.188982
传染科      0.176777
心胸外科     0.000000
肾内科      0.000000
老年科      0.000000
dtype: float64

In [58]:
zz2ks(['发烧','头晕'])

热咳 ['中医科'] 0.5773502691896258
恶性高血压视网膜病变 ['眼科', '心血管内科'] 0
恶寒发热 ['中医科'] 0.42640143271122083
房事头疼症 ['中医科'] 0.4082482904638631
胃穿孔 ['胃肠外科'] 0.4082482904638631
高温引起的溶血性贫血 ['血液科', '烧伤科'] 0.4082482904638631
疫黄 ['中医科'] 0.4082482904638631
伏暑 ['中医科'] 0.35355339059327373
镰状细胞贫血眼部病变 ['眼科', '血液科'] 0.35355339059327373
风温 ['中医科'] 0.35355339059327373
漆疮 ['中医科'] 0.35355339059327373
厥阴头痛 ['中医科'] 0.35355339059327373
电脑视觉综合症 ['眼科', '营养科'] 0.35355339059327373
扁桃体炎 ['耳鼻喉科', '呼吸内科'] 0.35355339059327373
虫证 ['中医科'] 0.15811388300841897
沙雷菌肺炎 ['呼吸内科'] 0.31622776601683794
药物中毒 ['急诊科', '消化内科'] 0.15811388300841897
杨梅疮 ['中医科', '性病科'] 0.31622776601683794
郁冒 ['中医科'] 0.31622776601683794
乳痈 ['中医科', '产科'] 0.31622776601683794
小儿乳积 ['儿科', '中医科'] 0
钩端螺旋体肺炎 ['呼吸内科'] 0.31622776601683794
血清病性荨麻疹 ['皮肤科'] 0.31622776601683794
阳虚发热 ['中医科'] 0.31622776601683794
药物相关性免疫性溶血性贫血 ['血液科'] 0.31622776601683794
砷化氢引起的溶血性贫血 ['血液科', '急诊科'] 0.2886751345948129
黄杆菌肺炎 ['呼吸内科'] 0.14433756729740646
胆囊炎 ['肝胆外科', '消化内科'] 0.2886751345948129
冷抗体型自身免疫性

中医科      8.431884
呼吸内科     3.558923
眼科       3.168702
血液科      2.626775
消化内科     1.343802
传染科      0.997259
耳鼻喉科     0.853553
性病科      0.801930
急诊科      0.798851
皮肤科      0.748802
肝胆外科     0.538675
胃肠外科     0.408248
烧伤科      0.408248
营养科      0.353553
口腔科      0.329124
产科       0.316228
风湿科      0.288675
肝炎       0.250000
心理咨询     0.250000
心血管内科    0.223607
儿科       0.204124
肾内科      0.182574
脑外科      0.133631
神经内科     0.133631
内分泌科     0.125000
体检保健科    0.110432
dtype: float64